# Investigating the Impacts of Urban Expansion in Phoenix, AZ
## Assessing Changes in the Biodiversity Intactness Index (BII) from 2017 to 2020
### Author: Carmen Hoyt
### [GitHub Repository](https://github.com/ceh58/biodiversity-phoenix)

### About
- Purpose
- Highlights
- Data
- References